# QuickStart по работе с Triton 

Данный ноутбук демонстрирует полный цикл обучения, конвертации модели и запуск инференса Triton.




#### Что такое NVIDIA Triton?
Triton Inference Server оптимизирует вывод ИИ, позволяя командам развертывать, запускать и масштабировать обученные модели ИИ из любой среды в любой инфраструктуре на основе графического процессора или процессора. Это дает исследователям искусственного интеллекта и специалистам по данным свободу выбора правильной платформы для своих проектов, не влияя на производственное развертывание


## Установка зависимостей 

In [ ]:
!pip install transformers
!pip install torch
!pip install datasets

In [ ]:
import transformers, torch, datasets
print("transformers", transformers.__version__)
print("torch", torch.__version__)
print("datasets", datasets.__version__)

## Набор данных

В этом примере используется датасет [emotion](https://huggingface.co/datasets/emotion). Этот датасет содержит набор сообщений из Twitter и размечен на 6 эмоций sadness (0), joy (1), love (2), anger (3), fear (4), surprise (5).

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

dataset = load_dataset("emotion")

## Предобработка 

Этот этап необходим для предобработки текстовых сообщений (конвертации текста в вектор)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_dataset = dataset.map(preprocess_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Обучение

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer


model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=6)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

# Инференс

Для удобства использования модели в инференсе, можно переименовать параметры с помощью словарей label2id и id2label. Это позволит при выводе результатов, видеть классы.

In [ ]:
from transformers import AutoConfig, AutoModelForSequenceClassification

label2id = {
    "sadness": 0,
    "joy": 1,
    "love": 2,
    "anger": 3,
    "fear": 4,
    "surprise": 5
  }
id2label = {
    0: "sadness",
    1: "joy",
    2: "love",
    3: "anger",
    4: "fear",
    5: "surprise"
  }
model_ckpt = "./results/checkpoint-5000"
config = AutoConfig.from_pretrained(model_ckpt, label2id=label2id, id2label=id2label)


In [ ]:
from transformers import DistilBertForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("./results/checkpoint-5000")
model = AutoModelForSequenceClassification.from_pretrained("./results/checkpoint-5000", config=config)

In [ ]:
text = "I am incredibly happy to start using Triton on ML-Space from SberCloud"

tensor = tokenizer(text, padding="max_length",  truncation=True, max_length=512, return_tensors="pt")

In [ ]:
print("Example output", model(**tensor))

In [ ]:
logits = model(**tensor).logits
predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

In [ ]:
del config
del model
del tokenizer

## Подготовка модели к инференсу на Triton


Для инференса модели на Triton необходимо PyTorch модель перевести в TorchScript. Для этой конвертации неоходимо показать модели пример входного и выходного вектора

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("./results/checkpoint-5000")

tensors = tokenizer(text, padding="max_length",  truncation=True, return_tensors='pt', max_length=512)
example_inputs = tensors['input_ids'], tensors['attention_mask']

In [ ]:
import torch

class PyTorch_to_TorchScript(torch.nn.Module):
    def __init__(self):
        super(PyTorch_to_TorchScript, self).__init__()
        self.model = AutoModelForSequenceClassification.from_pretrained("./results/checkpoint-5000")

    def forward(self,data, attention_mask=None):
        return self.model(data, attention_mask)

In [ ]:
pt_model = PyTorch_to_TorchScript().eval()

In [ ]:
scripted_model = torch.jit.trace(pt_model, [tensors['input_ids'], tensors['attention_mask']], strict=False)

In [ ]:
scripted_model.graph

In [ ]:
scripted_model(tensors['input_ids'], tensors['attention_mask'])

Перед сохранением модели необходимо создать каталог:

```
model_repository_path/
|- <pytorch_model_name>/
|  |- config.pbtxt
|  |- 1/
|     |- model.pt
|
```

Где **pytorch_model_name** - название модели, **config.pbtxt** - конфигурация для Triton, **model.pt** - экспортированная модель. Структура каталогов будет выглядеть так:

```
triton_inf/
|- / distil_bert_emotion
|  |- config.pbtxt
|  |- 1/
|     |- model.pt
|
```

In [ ]:
!mkdir Triton
!mkdir Triton/Predictor
!mkdir Triton/Predictor/distil_bert_emotion
!mkdir Triton/Predictor/distil_bert_emotion/1

In [ ]:
scripted_model.save('./Triton/Predictor/distil_bert_emotion/1/model.pt')

Теперь необходимо описать модель для Triton

Пример **config.pbtxt** 
```
name: "distil_bert_emotion"
platform: "pytorch_libtorch"
input [
 {
    name: "input__0"
    data_type: TYPE_INT32
    dims: [1, 512]
  } ,
{
    name: "input__1"
    data_type: TYPE_INT32
    dims: [1, 512]
  }
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [1, 6]
  }

instance_group [
    {
        count: 1
        kind: KIND_GPU
    }
]
```

Где поле **name** - наименование модели,  **input** - описывает входной массив модели, **output** - описывает выходной массив. 

**input** указываются входные вектора. В этом примере у нас два входных вектора *input_ids* и *attention_mask* каждый имеет размерность `[1,512]` и тип данных `int32`. 

**output** указывает выходной вектор. В этом примере выходной вектор `[1,6]` и формат fp32

Более подробно о написании **config.bptxt** можно ознакомиться в документации [Triton](https://github.com/triton-inference-server/server/blob/main/docs/model_configuration.md)

In [ ]:
%%bash
 
cat >> Triton/Predictor/distil_bert_emotion/config.pbtxt << EOF
name: "distil_bert_emotion"
platform: "pytorch_libtorch"
input [
 {
    name: "input_ids"
    data_type: TYPE_INT32
    dims: [1, 512]
  } ,
{
    name: "attention_mask"
    data_type: TYPE_INT32
    dims: [1, 512]
  }
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [1, 6]
  }

instance_group [
    {
        count: 1
        kind: KIND_GPU
    }
]
EOF

## Transformer-скрипт

Serving-скрипт отвечает за получение запроса, предобработку, отправку запроса в предиктор, постобработку предиктора.

Для предобработки используется AutoTokenizer, ему необходимо указать откуда загрузить токенизатор.

Для этого создадим директорию Transformer 

In [ ]:
!mkdir Triton/Transformer
!mkdir Triton/Transformer/tokenizer

In [ ]:
!cp results/checkpoint-5000/tokenizer.json Triton/Transformer/tokenizer
!cp results/checkpoint-5000/tokenizer_config.json Triton/Transformer/tokenizer
!cp results/checkpoint-5000/vocab.txt Triton/Transformer/tokenizer

In [ ]:
!ls -l Triton/Transformer/tokenizer

In [ ]:
# Файл Transformer/kf_serving.py

import re
import os
import argparse

import kfserving
from typing import Dict
import numpy as np
import tritonclient.http as httpclient
import logging
from transformers import AutoTokenizer

logging.basicConfig(level=logging.DEBUG)


class BertTransformer(kfserving.KFModel):
    def __init__(self, name: str, predictor_host: str):
        super().__init__(name)
        self.predictor_host = predictor_host
        # токенайзер с сохранеными файлами
        # из соображений безопастности доступа в интернет из контейнера деплоя нет
        self.tokenizer = AutoTokenizer.from_pretrained('./tokenizer') 
        # наименование модели (из configb.pbtxt)
        self.model_name = "distil_bert_emotion" 
        self.triton_client = None

    def preprocess(self, inputs: Dict) -> Dict:
        """
            Препроцесинг входных данных 
        """
         # токенезируем входной запрос
        tensors = self.tokenizer(inputs["instances"][0], padding="max_length",  truncation=True, return_tensors='pt', max_length=512)

        return {"input_ids":tensors['input_ids'], "attention_mask":tensors['attention_mask']}

    def predict(self, features: Dict) -> Dict:
        """
            Предикт     
        """
        if not self.triton_client:
            self.triton_client = httpclient.InferenceServerClient(
                url=self.predictor_host, verbose=True)

        input_ids = np.array(features['input_ids'], dtype=np.int32) # конвертируем вектор  в int32
        attention_mask = np.array(features['attention_mask'], dtype=np.int32) # конвертируем вектор  в int32

        input_ids = features["input_ids"].reshape(1, 512) # преобразуем в [1,512]
        attention_mask = features["attention_mask"].reshape(1, 521)  # преобразуем в [1,512]

        # Формируем запрос в тритон
        inputs = [httpclient.InferInput('input_ids', [1, 512], "INT32"), 
                  httpclient.InferInput('attention_mask', [1, 512], "INT32")]  
        # Заполняем запрос данными из numpy массива
        inputs[0].set_data_from_numpy(input_ids) 
        inputs[1].set_data_from_numpy(attention_mask)

        
        # Указываем ожидаемый выходной результат сети
        outputs = [httpclient.InferRequestedOutput('output__0', binary_data=False),] 
        result = self.triton_client.infer(self.model_name, inputs, outputs=outputs)
        return result.get_response()

    def postprocess(self, result: Dict) -> Dict:
        """
            Обработка результата сети
        """
        logging.info(result)
        prediction = result['outputs'][0]['data']

        return {"predictions": prediction}

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--http-port", default=8080)
    parser.add_argument("--predictor-host")
    args = parser.parse_args()

    x = re.compile('(kfserving-\d+)').search(os.environ.get('HOSTNAME'))
    name = "kfserving-default"
    if x:
        name = x[0]
    model = BertTransformer(name, predictor_host=args.predictor_host)
    kfserving.KFServer(workers=1, http_port=args.http_port).start([model])


Сформированный скрипт для сервинга модели необходимо сохранить по пути `Triton/Transformer/kf_serving.py`

Для работы kf_serving.py скрипта необходимо добавить в установку используемые в нем зависимости. 

In [ ]:
%%bash
 
cat >> Triton/Transformer/requirements.txt << EOF
tritonclient [all]
transformers
torch
numpy
EOF

Итоговая структура директории:
```
 |-Triton
 | |-Transformer
 | | |-tokenizer
 | | | |-tokenizer.json
 | | | |-tokenizer_config.json
 | | | |-vocab.txt
 | | |-requirements.txt
 | | |-kf_serving.py
 | |-Predictor
 | | |-distil_bert_emotion
 | | | |-1
 | | | | |-model.pt
 | | | |-config.pbtxt
```

## Создание образа

Для сборки образа необходимо созданные папки **Transformer** и **Predictor** загрузить в бакет S3. Если бакет создан, то нужно перейти в раздел получения credentials. Для создания бакета S3 Data Catalog -> Обзор Хранилища -> Создать Бакет.


<img src="img/data_storage.png" alt="drawing" width="200"/>

![data storage](img/storage.png)


После создания бакета необходимо получить его credentials для подключения с помощью сторонних утилит и последующей загрузки файлов. 


<img src="img/get_cred.png" alt="drawing" width="500"/>
<img src="img/view_cred.png" alt="drawing" width="500"/>


После того как получили credentials необходимо скопировать: 
- S3 endpoint
- S3 имя бакета
- S3 access key ID
- S3 security key

In [ ]:
import boto3
import os
from tqdm import tqdm

S3_ACCESS_KEY_ID = "ВАШ_S3_ACCESS_KEY_ID"
S3_SECRET_ACCESS_KEY_ID = "ВАШ_S3_SECRET_ACCESS_KEY_ID"
BUCKET_NAME = "ВАШ_BUCKET_NAME"
ENDPOINT_URL = "ВАШ_ENDPOINT_URL"

def upload_files(bucket, path):
    session = boto3.session.Session()
 
    s3_client = session.client(
        service_name='s3',
        aws_access_key_id=S3_ACCESS_KEY_ID,
        aws_secret_access_key=S3_SECRET_ACCESS_KEY_ID,
        endpoint_url=ENDPOINT_URL
    )
 
    for subdir, dirs, files in tqdm(os.walk(path)):
        for file in files:
            full_path = os.path.join(subdir, file)
            with open(full_path, 'rb') as data:
                s3_client.put_object(Bucket = bucket, Key=full_path[len(path)+1:], Body=data)


Загрузим каталоги из Triton в S3

In [ ]:
upload_files(BUCKET_NAME, './Triton')

После загрузки можем приступить к сборке образа. Для сборки образа зайти в Deployment->Образы и нажмите "Создать образ"


<img src="img/image.png" alt="drawing" width="900"/>


Первым образом, соберем "Трансформер".

1. Тип образа  - Triton Server
2. Тип контейнера - Трансформер
3. Базовый образ - cr.msk.sbercloud.ru/aicloud-base-images/triton22.04-py3:0.0.32 
4. Хранилище - тот S3 бакет в который загружали ранее 
5. Конфигурация
    - Папка с моделью -  Transformer 
    - Файл Serving-script - kf_serving.py
    - Файл Requirements - requirements.txt
    
<img src="img/image_build_transformer.png" alt="" width="900"/>

Вторым образом, соберем "Предиктор".

1. Тип образа  - Triton Server
2. Тип контейнера - Предиктор
3. Базовый образ - cr.msk.sbercloud.ru/aicloud-base-images/triton22.04-py3:0.0.32 
4. Хранилище - тот S3 бакет в который загружали ранее 
5. Конфигурация
    - Папка с файлами конфигурации - папка с моделью. Пример - ИМЯ_БАКЕТА/Predictor


<img src="img/image_build_predictor.png" alt="" width="900"/>



## Деплой


Для деплоя модели зайдите в Deployment -> Деплои


<img src="img/deploi.png" alt="" width="900"/>

Нажмите кнопку "Создать деплой". Укажите следующие настройки. 

1. Наименование - Название сборки (можно оставить пустым)
2. Тип деплоя - Раздельный
3. Ресурсы - указываем регион и тип конфигурации 
4. Указываем долю ресурсов от общей конфигурации для контейнера Transformer
5. Выберите Docker-образ - указываете собранные Docker собранные ранее 


<img src="img/create_deploi.png" alt="" width="900"/>



После создания, появится карточка с созданным деплоем, со статусом **"В очереди"**. То есть данный деплой находиться на стадии ожидания выбранных ресурсов и как только ресурсы станут доступны, деплой передает в статус **"Выполняется"**

Обратите внимание, что если минимальное количество Pods будет установлено в "0", то горячий деплой не будет запущен сразу. В таком случае при первом запросе, вы получите дополнительную задержку на поднятии деплоя. 


Открыв карточку с запущеным деплоем можно посмотреть и изменить текущую конфигурацию.

<img src="img/image_triron.png" alt="" width="900"/>

Так же можно отправить тестовый запрос из вкладки "Тест API" и скопировать его в виде cURL 

<img src="img/image_example_requests.png" alt="" width="900"/>

Во вкладке "Логи" можно посмотреть текущее состояние деплоя Triton 

<img src="img/example_logs.png" alt="" width="900"/>
